In [8]:
from core import *

In [9]:
param_grid = {
    "window_size": ['suss'],
    "distance": ['euclidean_distance'],
    "score":['roc_auc','f1'],
    "early_stopping": [True,False],
    "k_neighbours":[1,3,5,7,9],
    "n_jobs":[4]
}
for p_idx,params in enumerate(ParameterGrid(param_grid)):
    print(f'idx:{p_idx} {params}')

idx:0 {'distance': 'euclidean_distance', 'early_stopping': True, 'k_neighbours': 1, 'n_jobs': 4, 'score': 'roc_auc', 'window_size': 'suss'}
idx:1 {'distance': 'euclidean_distance', 'early_stopping': True, 'k_neighbours': 1, 'n_jobs': 4, 'score': 'f1', 'window_size': 'suss'}
idx:2 {'distance': 'euclidean_distance', 'early_stopping': True, 'k_neighbours': 3, 'n_jobs': 4, 'score': 'roc_auc', 'window_size': 'suss'}
idx:3 {'distance': 'euclidean_distance', 'early_stopping': True, 'k_neighbours': 3, 'n_jobs': 4, 'score': 'f1', 'window_size': 'suss'}
idx:4 {'distance': 'euclidean_distance', 'early_stopping': True, 'k_neighbours': 5, 'n_jobs': 4, 'score': 'roc_auc', 'window_size': 'suss'}
idx:5 {'distance': 'euclidean_distance', 'early_stopping': True, 'k_neighbours': 5, 'n_jobs': 4, 'score': 'f1', 'window_size': 'suss'}
idx:6 {'distance': 'euclidean_distance', 'early_stopping': True, 'k_neighbours': 7, 'n_jobs': 4, 'score': 'roc_auc', 'window_size': 'suss'}
idx:7 {'distance': 'euclidean_dista

In [10]:
with open("../dfl.pkl", "rb") as f:
    dfl = pickle.load(f)

with open("../gtl.pkl", "rb") as f:
    gtl = pickle.load(f)

In [11]:
#1. model setting: already computed with given upper hyperparameters
modelpred=[]
for i in range(0,20):
    with open("../cpsl_param_"+str(i)+".pkl", "rb") as f:
        modelpred.append(pickle.load(f))

#20 model
#21 video
#98 feature

In [12]:
#2. splitting
def SplitSaliency(pred):
    # 2. split features in 6 saliency groups
    # vel,acc,jerk,specific,generic
    # specific: other type of feature specific for that saliency
    # generic: other type of feature that describe the whole body movement
    ## generic features: 0,3
    ariel_el=[26,27,28,29,30,31,32,33,34,35,36,37,2,8,14,0,3]
    ariel = UnionCPS(*pred[ariel_el])
   
    strn_el=[38,39,40,41,42,43,44,45,46,47,48,49,1,9,0,3]
    strn = UnionCPS(*pred[strn_el])

    rhel_el=[50,51,52,53,54,55,56,57,58,59,60,61,7,11,16,17,18,21,0,3]
    rhel = UnionCPS(*pred[rhel_el])

    lhel_el=[62,63,64,65,66,67,68,69,70,71,72,73,6,10,16,17,19,20,0,3]
    lhel = UnionCPS(*pred[lhel_el])
    
    rplm_el=[74,75,76,77,78,79,80,81,82,83,84,85,5,13,15,23,25,0,3]
    rplm = UnionCPS(*pred[rplm_el])

    lplm_el=[86,87,88,89,90,91,92,93,94,95,96,97,4,12,15,22,24,0,3]
    lplm = UnionCPS(*pred[lplm_el])

    return [ariel,strn,rhel,lhel,rplm,lplm]

In [13]:
# 3.
def CombineSingleSaliency(saliencies, maj_margin, maj_thre):
    res=[]
    for sal in saliencies:
        res.append(MajorityVoteCP(sal, maj_margin, maj_thre))
    return res
        


In [14]:

#98 per un solo video

# 1. model setting - in: 98 timeseries - out: 98 prediction
# 2. splittare in 6 salienze - in: 98 prediction - out: 98 into 6 list[salienza][feature][elemento singolo] (could be repetition)
# 3xyz. majvote(threshold,margin) - in: 6 list[][][] - out: 6 list[salienza][elemento singolo]
# 3tog. majvote(threshold,margin) - in: 6 list[][][] - out: 6 list[salienza][elemento singolo]
# 4. unione - in: 6 list[][] - out: list[]
# 5. ignorezone(margin)
# 6. delnear(margin)
# 7. dtw(threshold)
def PostFilterTogether(model_id,video_id,param_id,param_len,density,pred,gt, maj_margin, maj_thre, zone_margin, del_margin, dtw_threshold):
    """
        pred: 98 predictions of a single video
    """
    
    print(f"model:{model_id}/20 - video:{video_id}/21 - param:{param_id}/{param_len} - Start                                               ", end='\r')

    #2.
    saliencies_raw = SplitSaliency(pred)
    print(f"model:{model_id}/20 - video:{video_id}/21 - param:{param_id}/{param_len} - Split Saliency completed                            ", end='\r')
    #3.
    saliencies = CombineSingleSaliency(saliencies_raw,maj_margin,maj_thre)
    print(f"model:{model_id}/20 - video:{video_id}/21 - param:{param_id}/{param_len} - Combine Saliency completed                          ", end='\r')
    #4.
    pred_united = UnionCP(saliencies)
    print(f"model:{model_id}/20 - video:{video_id}/21 - param:{param_id}/{param_len} - Union completed                                     ", end='\r')
    #5.
    pred_ignored = IgnoreZone(video_id,pred_united,gt,zone_margin)
    print(f"model:{model_id}/20 - video:{video_id}/21 - param:{param_id}/{param_len} - Ignore Zone completed                               ", end='\r')
    #6.
    pred_del = delnear(pred_ignored,del_margin)
    print(f"model:{model_id}/20 - video:{video_id}/21 - param:{param_id}/{param_len} - delnear completed                                   ", end='\r')
    #7.
    pred_final=FPremoverDTW(density,pred_del,dtw_threshold)
    print(f"model:{model_id}/20 - video:{video_id}/21 - param:{param_id}/{param_len} - DTW completed                                       ", end='\r')
    # pred_modelid_videoid_majmargin_majthree_zonemargin_delmargin_dtwthreshold
    return pred_final
    



In [15]:
#leave one video out

param_grid = {
    "maj_margin": [80,100,120],
    "maj_thre": [1,2,3],
    "zone_margin":[80,100,120],
    "del_margin": [80,100,120],
    "dtw_threshold":[80,100,120]
}



# 21*98 = 2058 timeseries
model_res=[]
for model_id,model in enumerate(modelpred): #20 models predictions
    param_value=[]
    for p_idx,params in enumerate(ParameterGrid(param_grid)):
        video=[]
        for i in range(len(TIMESERIES)): #21 videos
            df = dfl[i]
            gt = gtl[i]
            res=PostFilterTogether(model_id,i,p_idx,len(ParameterGrid(param_grid)),df.iloc[:,3],model[i],gt,params["maj_margin"],params["maj_thre"],params["zone_margin"],params["del_margin"],params["dtw_threshold"])
            video.append(res)
            """
            i folds servono solo durante lo scoring
            # single fold
            train_set = [j for j in range(len(TIMESERIES)) if j != i]
            test_set = i
            for ts in train_set:
                PostFilterTogether()
            """
        param_value.append(video)
    model_res.append(param_value)

with open("../pred_filtered/collection_models_prediction_filtered.pkl", "wb") as f:
    pickle.dump(model_res, f)
    
    #fuori dal for deve calcolare la media
    
"""
versione 1
fisso param_grid
calcolo f1 score per only training fold
prendo il migliore
calcolo per ogni singolo video nel validation

for model in models:
    for param in paramgrid:
        for fold in folds:
            for ts in fold.train:
                f1s_1 = compute single f1score 
                calcolo f1score per singolo ts
            f1s_mean = compute mean f1score mean(f1s_1,f1s_1,...)
            per un intero train_set ritorno lo il f1score medio
        f1s_mean_mean = compute mean of f1s_mean(f1s_mean,f1s_mean,...)
        per ogni train_set ritorno la media = come performa questo param across all folds
    check best f1s_mean_mean
per ogni param diverso, controllo quello che mi massimizza
O(n^3)
"""





TypeError: list indices must be integers or slices, not list